## 재무비율 전처리
1. 모든 컬럼에 값이 공통으로 없는 행 -> 개별재무제표 값으로 처리
2. 전체에 대해서 값이 없는 컬럼 삭제

In [1]:
import pandas as pd

kospi_ratio = pd.read_csv('../data/processed_data/재무비율.csv')
kospi_ind = pd.read_excel('../data/raw_data/코스피_개별재무비율.xlsx')
kosdaq_ind = pd.read_excel('../data/raw_data/코스닥_개별재무비율.xlsx')

In [2]:
listed = pd.concat([kospi_ind, kosdaq_ind], ignore_index=True)

listed['회계년도'] = pd.to_datetime(listed['회계년도'], format='%Y/%m')
listed = listed[listed['회계년도'].dt.month == 12]

---
### 1. 모든 컬럼에 값이 공통으로 없는 행 -> 개별재무제표 값으로 처리

#### 전체에 대해서 값이 없는 행(기업) 탐색

In [3]:
# 모든 컬럼에서 값이 없는(결측치) 행을 찾기
empty_rows = kospi_ratio[kospi_ratio.iloc[:, 9:-1].isnull().all(axis=1)]

print(len(empty_rows.drop_duplicates('회사명')['회사명']))
print('------------------------------------------')
print(empty_rows.drop_duplicates('회사명')['회사명'])

35
------------------------------------------
7          (주)대호에이엘
12       (주)신화다이나믹스
74          백광산업(주)
78      삼성바이오로직스(주)
95        코스모신소재(주)
98         코스모화학(주)
105       한국전력기술(주)
114      한솔아트원제지(주)
131           (주)광무
144          (주)뉴보텍
156        (주)디지털대성
158          (주)메지온
165           (주)삼일
167           (주)상보
173         (주)서울제약
195          (주)시큐브
211           (주)씨젠
227        (주)에스에스알
246         (주)와이어블
274      (주)제이테크놀로지
286     (주)케이바이오컴퍼니
291    (주)케이에스인더스트리
296     (주)케이지모빌리언스
298      (주)케이지이니시스
306        (주)타이거일렉
309       (주)티에스넥스젠
328        (주)한진피앤씨
334         경남제약(주)
339       대한그린파워(주)
341          더라미(주)
347         디에스티(주)
350        삼영이엔씨(주)
356        씨앤티85(주)
369       우양에이치씨(주)
380       한솔아이원스(주)
Name: 회사명, dtype: object


In [4]:
print(empty_rows.shape)
print(empty_rows.drop_duplicates('회사명').shape)

(89, 157)
(35, 157)


In [5]:
# empty_rows.to_csv('재무비율null.csv',index=False, encoding='UTF-8-sig')

In [6]:
# 결측치 기업들 feature 컬럼 제외
df = empty_rows.iloc[:, :9]
df['label'] = empty_rows['label']

df.reset_index(drop=True, inplace=True)
df

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,label
0,(주)대호에이엘,69460,2014,1,24,32402,1차 금속 제조업,2002-11-11,NaN,1
1,(주)신화다이나믹스,1770,2012,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1
2,(주)신화다이나믹스,1770,2013,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1
3,(주)신화다이나믹스,1770,2014,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1
4,(주)신화다이나믹스,1770,2015,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1
...,...,...,...,...,...,...,...,...,...,...
84,한솔아이원스(주),114810,2014,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,1
85,한솔아이원스(주),114810,2015,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,1
86,한솔아이원스(주),114810,2016,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,1
87,한솔아이원스(주),114810,2017,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,1


---
#### 결측치 기업 개별재무제표 값으로 대체

In [7]:
df['회계년도'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 89 entries, 0 to 88
Series name: 회계년도
Non-Null Count  Dtype
--------------  -----
89 non-null     int64
dtypes: int64(1)
memory usage: 840.0 bytes


In [8]:
listed['회계년도'].info()

<class 'pandas.core.series.Series'>
Index: 27584 entries, 0 to 28141
Series name: 회계년도
Non-Null Count  Dtype         
--------------  -----         
27584 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 431.0 KB


In [9]:
listed['회계년도'] = listed['회계년도'].astype(str).str[:4].astype(int)

# 상장일 컬럼의 날짜 형식을 변경
listed['상장일'] = listed['상장일'].str.replace('/', '-')

In [10]:
check = pd.merge(df, listed, on = ['회사명', '거래소코드', '회계년도', '소속코드', '통계청 한국표준산업분류 코드 10차(대분류)',
                                   '산업코드', '산업명', '상장일', '상장폐지일'], how = 'left' )

check

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,label,...,세금과공과(IFRS)(백만원),감가상각비(IFRS)(백만원),종업원1인당 부가가치(IFRS)(백만원).1,총자본투자효율(IFRS).1,기계투자효율(IFRS).1,부가가치율(IFRS).1,종업원수(IFRS),"매출채권 대 상,제품비율(IFRS)","상품,제품회전률(IFRS)","원,부재료회전률(IFRS)"
0,(주)대호에이엘,69460,2014,1,24,32402,1차 금속 제조업,2002-11-11,NaN,1,...,0.0,3738.89,23.95,3.76,31.80,3.15,173.0,NaN,NaN,NaN
1,(주)신화다이나믹스,1770,2012,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1,...,0.0,1032.16,87.75,7.99,192.83,8.01,85.0,NaN,NaN,NaN
2,(주)신화다이나믹스,1770,2013,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1,...,0.0,1016.51,47.93,5.00,127.21,5.29,85.0,NaN,NaN,NaN
3,(주)신화다이나믹스,1770,2014,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1,...,0.0,975.52,85.39,9.65,285.26,11.57,84.0,NaN,NaN,NaN
4,(주)신화다이나믹스,1770,2015,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1,...,0.0,866.12,98.24,12.00,427.71,14.60,85.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,한솔아이원스(주),114810,2014,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,1,...,0.0,5874.10,80.59,24.18,64.97,36.25,311.0,156.50,9.75,150.39
85,한솔아이원스(주),114810,2015,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,1,...,0.0,4275.95,64.50,17.79,126.87,33.75,363.0,113.88,7.58,74.71
86,한솔아이원스(주),114810,2016,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,1,...,0.0,6968.28,31.26,7.78,47.27,14.45,393.0,93.62,5.48,70.98
87,한솔아이원스(주),114810,2017,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,1,...,0.0,7923.59,61.45,17.01,121.67,24.62,519.0,67.60,5.67,53.95


In [11]:
check.columns

Index(['회사명', '거래소코드', '회계년도', '소속코드', '통계청 한국표준산업분류 코드 10차(대분류)', '산업코드',
       '산업명', '상장일', '상장폐지일', 'label',
       ...
       '세금과공과(IFRS)(백만원)', '감가상각비(IFRS)(백만원)', '종업원1인당 부가가치(IFRS)(백만원).1',
       '총자본투자효율(IFRS).1', '기계투자효율(IFRS).1', '부가가치율(IFRS).1', '종업원수(IFRS)',
       '매출채권 대 상,제품비율(IFRS)', '상품,제품회전률(IFRS)', '원,부재료회전률(IFRS)'],
      dtype='object', length=160)

In [12]:
# label 컬럼을 분리
label_column = check.pop('label')

# 분리한 label 컬럼을 맨 마지막에 추가
check['label'] = label_column

In [13]:
check.head(1)

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,총자본증가율(IFRS),...,감가상각비(IFRS)(백만원),종업원1인당 부가가치(IFRS)(백만원).1,총자본투자효율(IFRS).1,기계투자효율(IFRS).1,부가가치율(IFRS).1,종업원수(IFRS),"매출채권 대 상,제품비율(IFRS)","상품,제품회전률(IFRS)","원,부재료회전률(IFRS)",label
0,(주)대호에이엘,69460,2014,1,24,32402,1차 금속 제조업,2002-11-11,NaN,-0.9,...,3738.89,23.95,3.76,31.8,3.15,173.0,NaN,NaN,NaN,1


In [14]:
# check.to_csv('재무비율_개별로대체.csv',index=False,encoding='UTF-8-sig')

---
#### 개별로 채워도 null값이 있는 컬럼

In [15]:
# 1개라도 null값이 있는 컬럼

check.columns[check.isnull().sum() > 0]

Index(['상장폐지일', '매출채권 대 상', '상품', '원', '매출채권 대 상,제품비율(IFRS)', '상품,제품회전률(IFRS)',
       '원,부재료회전률(IFRS)'],
      dtype='object')

In [16]:
check.columns[:157] == kospi_ratio.columns[:159]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [17]:
def clean_column_name(col_name):
    # col_name = col_name.replace('(*)', '').replace('(원)', '').replace('(백만원)', '')
    # col_name = col_name.replace('[제조]', '')
    # col_name = col_name.replace('(천원)', '')
    col_name = col_name.replace('(IFRS연결)', '(IFRS)')
    # col_name = col_name.replace('.1', '')

    return col_name.strip()  


kospi_ratio.columns = [clean_column_name(col) for col in kospi_ratio.columns]
kospi_ratio

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,총자본증가율(IFRS),...,금융비용(IFRS)(백만원),임차료(IFRS)(백만원),세금과공과(IFRS)(백만원),감가상각비(IFRS)(백만원),종업원1인당 부가가치(IFRS)(백만원).1,총자본투자효율(IFRS).1,기계투자효율(IFRS).1,부가가치율(IFRS).1,종업원수(IFRS),label
0,(주)다이나믹디자인,145210,2015,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,-4.56,...,3589.40,0.0,0.0,14203.75,NaN,19.72,98.91,29.06,NaN,1
1,(주)다이나믹디자인,145210,2016,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,-12.91,...,0.00,0.0,0.0,0.00,NaN,-3.89,0.00,-5.57,NaN,1
2,(주)다이나믹디자인,145210,2017,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,-17.07,...,3990.88,0.0,0.0,11694.91,NaN,8.59,44.66,12.60,NaN,1
3,(주)대우건설,47040,2012,1,41,64101,종합 건설업,2001-03-23,NaN,4.13,...,94086.89,124047.0,54145.0,51212.00,NaN,13.73,2422.60,16.47,NaN,0
4,(주)대우건설,47040,2013,1,41,64101,종합 건설업,2001-03-23,NaN,2.67,...,86015.38,123239.0,62921.0,59003.00,NaN,4.00,731.03,4.62,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,한솔아이원스(주),114810,2017,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
385,한솔아이원스(주),114810,2018,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
386,한솔아이원스(주),114810,2019,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,0.00,...,3425.26,0.0,0.0,8607.60,NaN,20.66,165.10,32.15,NaN,1
387,한솔아이원스(주),114810,2020,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,-12.56,...,2408.62,0.0,0.0,7715.50,NaN,27.08,192.48,30.81,NaN,1


In [18]:
check.columns[:157] == kospi_ratio.columns[:157]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,

In [19]:
# A와 B 데이터프레임이 있다고 가정
kospi_ratio_columns = kospi_ratio.columns.tolist()  # A 데이터프레임의 컬럼 이름 리스트
check_columns = check.columns.tolist()  # B 데이터프레임의 컬럼 이름 리스트

# 빈 데이터프레임 생성
df_comparison = pd.DataFrame(columns=['kospi_ratio_columns', 'check_columns'])

# 빈 데이터프레임에 A와 B의 컬럼 값을 넣기
df_comparison['kospi_ratio_columns'] = pd.Series(kospi_ratio_columns)
df_comparison['check_columns'] = pd.Series(check_columns)

# 결과 출력
df_comparison

,kospi_ratio_columns,check_columns
0,회사명,회사명
1,거래소코드,거래소코드
2,회계년도,회계년도
3,소속코드,소속코드
4,통계청 한국표준산업분류 코드 10차(대분류),통계청 한국표준산업분류 코드 10차(대분류)
...,...,...
152,총자본투자효율(IFRS).1,총자본투자효율(IFRS).1
153,기계투자효율(IFRS).1,기계투자효율(IFRS).1
154,부가가치율(IFRS).1,부가가치율(IFRS).1
155,종업원수(IFRS),종업원수(IFRS)


- 85 매출채권 대 상,제품비율(IFRS) vs 매출채권 대 상,
- 118 상품,제품회전률(IFRS) vs 상품
- 119 원,부재료회전률(IFRS) vs 원
- 156 label vs 매출채권 대 상,제품비율(IFRS)
- 157 X vs 상품,제품회전률(IFRS)
- 158 X vs 원,부재료회전률(IFRS)


개별재무비율로 채운 데이터셋 
- '매출채권 대 상,' 컬럼과 '매출채권 대 상,제품비율(IFRS)'에서 같은 값을 기업에 따라 각각 제출한듯
- '상품' 컬럼과 '상품,제품회전률(IFRS)'에서 같은 값을 기업에 따라 각각 제출한듯
- '원' 컬럼과 '원,부재료회전률(IFRS)'에서 같은 값을 기업에 따라 각각 제출한듯

In [20]:
len(kospi_ratio.columns)

157

In [21]:
len(check.columns)

160

In [22]:
# '매출채권 대 상'과 '매출채권 대 상,제품비율(IFRS)' 컬럼을 합쳐서 '매출채권 대 상,제품비율(IFRS)'로 업데이트

check['매출채권 대 상'] = check['매출채권 대 상'].combine_first(check['매출채권 대 상,제품비율(IFRS)'])

# 중복된 뒤에 컬럼은 삭제
check.drop(columns=['매출채권 대 상,제품비율(IFRS)'], inplace = True)

# 이름은 다시 제대로 된 이름으로 수정
check.rename(columns={'매출채권 대 상' : '매출채권 대 상,제품비율(IFRS)'}, inplace = True)
check['매출채권 대 상,제품비율(IFRS)'].isnull().sum()

0

In [23]:
len(check.columns)

159

In [24]:
# 남은 2개도 똑같이 처리

check['상품'] = check['상품'].combine_first(check['상품,제품회전률(IFRS)'])
check['원'] = check['원'].combine_first(check['원,부재료회전률(IFRS)'])

# 중복된 뒤에 컬럼은 삭제
check.drop(columns=['상품,제품회전률(IFRS)', '원,부재료회전률(IFRS)'], inplace = True)

# 이름은 다시 제대로 된 이름으로 수정
check.rename(columns={'상품' : '상품,제품회전률(IFRS)', '원' : '원,부재료회전률(IFRS)'}, inplace = True)
print(check['상품,제품회전률(IFRS)'].isnull().sum())
print(check['원,부재료회전률(IFRS)'].isnull().sum())

0
0


In [25]:
len(check.columns)

157

In [26]:
check.isnull().sum()

회사명                         0
거래소코드                       0
회계년도                        0
소속코드                        0
통계청 한국표준산업분류 코드 10차(대분류)    0
                           ..
총자본투자효율(IFRS).1             0
기계투자효율(IFRS).1              0
부가가치율(IFRS).1               0
종업원수(IFRS)                  0
label                       0
Length: 157, dtype: int64

In [27]:
# A와 B 데이터프레임이 있다고 가정
kospi_ratio_columns = kospi_ratio.columns.tolist()  # A 데이터프레임의 컬럼 이름 리스트
check_columns = check.columns.tolist()  # B 데이터프레임의 컬럼 이름 리스트

# 빈 데이터프레임 생성
df_comparison = pd.DataFrame(columns=['kospi_ratio_columns', 'check_columns'])

# 빈 데이터프레임에 A와 B의 컬럼 값을 넣기
df_comparison['kospi_ratio_columns'] = pd.Series(kospi_ratio_columns)
df_comparison['check_columns'] = pd.Series(check_columns)

# 결과 출력
df_comparison

,kospi_ratio_columns,check_columns
0,회사명,회사명
1,거래소코드,거래소코드
2,회계년도,회계년도
3,소속코드,소속코드
4,통계청 한국표준산업분류 코드 10차(대분류),통계청 한국표준산업분류 코드 10차(대분류)
...,...,...
152,총자본투자효율(IFRS).1,총자본투자효율(IFRS).1
153,기계투자효율(IFRS).1,기계투자효율(IFRS).1
154,부가가치율(IFRS).1,부가가치율(IFRS).1
155,종업원수(IFRS),종업원수(IFRS)


---
#### kospi_ratio에서 개별로 채운 기업들 삭제 후 concat

In [28]:
# 값이 없는 행 삭제
kospi_ratio_cleaned = kospi_ratio.drop(empty_rows.index)
kospi_ratio_cleaned

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,총자본증가율(IFRS),...,금융비용(IFRS)(백만원),임차료(IFRS)(백만원),세금과공과(IFRS)(백만원),감가상각비(IFRS)(백만원),종업원1인당 부가가치(IFRS)(백만원).1,총자본투자효율(IFRS).1,기계투자효율(IFRS).1,부가가치율(IFRS).1,종업원수(IFRS),label
0,(주)다이나믹디자인,145210,2015,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,-4.56,...,3589.40,0.00,0.00,14203.75,NaN,19.72,98.91,29.06,NaN,1
1,(주)다이나믹디자인,145210,2016,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,-12.91,...,0.00,0.00,0.00,0.00,NaN,-3.89,0.00,-5.57,NaN,1
2,(주)다이나믹디자인,145210,2017,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,-17.07,...,3990.88,0.00,0.00,11694.91,NaN,8.59,44.66,12.60,NaN,1
3,(주)대우건설,47040,2012,1,41,64101,종합 건설업,2001-03-23,NaN,4.13,...,94086.89,124047.00,54145.00,51212.00,NaN,13.73,2422.60,16.47,NaN,0
4,(주)대우건설,47040,2013,1,41,64101,종합 건설업,2001-03-23,NaN,2.67,...,86015.38,123239.00,62921.00,59003.00,NaN,4.00,731.03,4.62,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,참존글로벌(주),158310,2018,4,29,32902,기타 기계 및 장비 제조업,2013-04-03,2022/11/09,36.55,...,846.49,0.00,0.00,316.32,NaN,-13.80,-16451.76,-125.42,NaN,1
379,퀀타피아(주),78940,2018,6,46,74604,도매 및 상품 중개업,2004-08-27,NaN,-20.54,...,1364.41,119.33,221.42,1458.23,NaN,-22.07,-2398.22,-42.05,NaN,1
386,한솔아이원스(주),114810,2019,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,0.00,...,3425.26,0.00,0.00,8607.60,NaN,20.66,165.10,32.15,NaN,1
387,한솔아이원스(주),114810,2020,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,-12.56,...,2408.62,0.00,0.00,7715.50,NaN,27.08,192.48,30.81,NaN,1


In [29]:
# kospi_ratio_cleaned와 check를 위아래로 연결
combined_df = pd.concat([kospi_ratio_cleaned, check], ignore_index=True)
combined_df

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,총자본증가율(IFRS),...,금융비용(IFRS)(백만원),임차료(IFRS)(백만원),세금과공과(IFRS)(백만원),감가상각비(IFRS)(백만원),종업원1인당 부가가치(IFRS)(백만원).1,총자본투자효율(IFRS).1,기계투자효율(IFRS).1,부가가치율(IFRS).1,종업원수(IFRS),label
0,(주)다이나믹디자인,145210,2015,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,-4.56,...,3589.40,0.0,0.0,14203.75,NaN,19.72,98.91,29.06,NaN,1
1,(주)다이나믹디자인,145210,2016,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,-12.91,...,0.00,0.0,0.0,0.00,NaN,-3.89,0.00,-5.57,NaN,1
2,(주)다이나믹디자인,145210,2017,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,-17.07,...,3990.88,0.0,0.0,11694.91,NaN,8.59,44.66,12.60,NaN,1
3,(주)대우건설,47040,2012,1,41,64101,종합 건설업,2001-03-23,NaN,4.13,...,94086.89,124047.0,54145.0,51212.00,NaN,13.73,2422.60,16.47,NaN,0
4,(주)대우건설,47040,2013,1,41,64101,종합 건설업,2001-03-23,NaN,2.67,...,86015.38,123239.0,62921.0,59003.00,NaN,4.00,731.03,4.62,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,한솔아이원스(주),114810,2014,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,-0.08,...,1153.62,0.0,0.0,5874.10,80.59,24.18,64.97,36.25,311.0,1
385,한솔아이원스(주),114810,2015,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,26.94,...,986.84,0.0,0.0,4275.95,64.50,17.79,126.87,33.75,363.0,1
386,한솔아이원스(주),114810,2016,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,19.93,...,3567.48,0.0,0.0,6968.28,31.26,7.78,47.27,14.45,393.0,1
387,한솔아이원스(주),114810,2017,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,18.82,...,5048.10,0.0,0.0,7923.59,61.45,17.01,121.67,24.62,519.0,1


In [30]:
combined_df.sort_values(by=['거래소코드', '회계년도'], ascending=[True, True])

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,총자본증가율(IFRS),...,금융비용(IFRS)(백만원),임차료(IFRS)(백만원),세금과공과(IFRS)(백만원),감가상각비(IFRS)(백만원),종업원1인당 부가가치(IFRS)(백만원).1,총자본투자효율(IFRS).1,기계투자효율(IFRS).1,부가가치율(IFRS).1,종업원수(IFRS),label
101,현대건설(주),720,2013,1,41,64102,종합 건설업,1984-12-22,NaN,15.06,...,157.00,0.00,0.00,93263.00,NaN,13.79,389.30,14.60,NaN,1
102,현대건설(주),720,2014,1,41,64102,종합 건설업,1984-12-22,NaN,24.40,...,31992.00,0.00,0.00,154054.00,NaN,13.15,414.72,13.89,NaN,1
103,현대건설(주),720,2015,1,41,64102,종합 건설업,1984-12-22,NaN,6.06,...,29408.00,0.00,0.00,195819.00,NaN,14.03,452.89,14.12,NaN,1
104,현대건설(주),720,2016,1,41,64102,종합 건설업,1984-12-22,NaN,2.70,...,6913.00,0.00,0.00,190461.00,NaN,13.96,412.52,14.74,NaN,1
308,백광산업(주),1340,2015,1,20,32001,화학물질 및 화학제품 제조업; 의약품 제외,1976-06-10,NaN,-31.06,...,3179.30,955.06,0.00,19990.25,204.45,7.92,52.38,18.99,120.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,(주)씨엠에스에듀,225330,2016,4,85,168505,교육 서비스업,2016-04-07,2022/03/17,123.95,...,-77.89,4832.26,346.33,1773.32,NaN,34.09,540.44,39.08,NaN,1
168,(주)씨엠에스에듀,225330,2017,4,85,168505,교육 서비스업,2016-04-07,2022/03/17,9.96,...,-346.00,5965.99,493.86,2982.47,NaN,39.66,419.82,43.79,NaN,1
344,(주)에스에스알,275630,2018,5,58,105802,출판업,2018-08-06,NaN,0.00,...,-43.93,0.00,0.00,693.64,60.95,23.80,0.00,51.01,101.0,1
345,(주)에스에스알,275630,2019,5,58,105802,출판업,2018-08-06,NaN,2.29,...,-227.10,0.00,0.00,858.04,61.66,24.93,0.00,50.93,107.0,1


---
### 2. 전체에 대해서 값이 없는 컬럼 삭제

In [31]:
# 값이 아예 없는(전체 결측치인) 컬럼 찾기
empty_columns = combined_df.columns[combined_df.isnull().all()]
print(empty_columns)
print(len(empty_columns))

Index([], dtype='object')
0


In [32]:
# 연결재무제표에서 값이 아예 없는(전체 결측치인) 컬럼 찾기
empty_columns = kospi_ratio.columns[kospi_ratio.isnull().all()]
print(empty_columns)
print(len(empty_columns))

Index(['종업원1인당 부가가치증가율(IFRS)', '종업원수증가율(IFRS)', '종업원1인당 매출액증가율(IFRS)',
       '종업원1인당 인건비증가율(IFRS)', '사내유보율(IFRS)', '사내유보 대 자기자본비율(IFRS)',
       '평균배당률(IFRS)', '자기자본배당률(IFRS)', '배당성향(IFRS)', '1주당순자산(IFRS)(원)',
       '종업원1인당 부가가치(IFRS)(백만원)', '종업원1인당 매출액(IFRS)(백만원)',
       '종업원1인당 정상영업이익(IFRS)(백만원)', '종업원1인당 순이익(IFRS)(백만원)',
       '종업원1인당 인건비(IFRS)(백만원)', '노동장비율(IFRS)', '기계장비율(IFRS)', '자본집약도(IFRS)',
       '종업원1인당 부가가치(IFRS)(백만원).1', '종업원수(IFRS)'],
      dtype='object')
20


개별로 가져온값들은 위의 값들을 가지고 있음

In [33]:
combined_df.columns

Index(['회사명', '거래소코드', '회계년도', '소속코드', '통계청 한국표준산업분류 코드 10차(대분류)', '산업코드',
       '산업명', '상장일', '상장폐지일', '총자본증가율(IFRS)',
       ...
       '금융비용(IFRS)(백만원)', '임차료(IFRS)(백만원)', '세금과공과(IFRS)(백만원)',
       '감가상각비(IFRS)(백만원)', '종업원1인당 부가가치(IFRS)(백만원).1', '총자본투자효율(IFRS).1',
       '기계투자효율(IFRS).1', '부가가치율(IFRS).1', '종업원수(IFRS)', 'label'],
      dtype='object', length=157)

In [34]:
check2 = pd.merge(combined_df, listed, on = ['회사명', '거래소코드', '회계년도', '소속코드', '통계청 한국표준산업분류 코드 10차(대분류)',
                                   '산업코드', '산업명', '상장일', '상장폐지일'], how = 'left' )

In [35]:
print(len(combined_df))
print(len(check2))

389
389


In [36]:
check2.isnull().sum()

회사명                           0
거래소코드                         0
회계년도                          0
소속코드                          0
통계청 한국표준산업분류 코드 10차(대분류)      0
                           ... 
부가가치율(IFRS).1_y               0
종업원수(IFRS)_y                  0
매출채권 대 상,제품비율(IFRS)_y       128
상품,제품회전률(IFRS)_y            128
원,부재료회전률(IFRS)_y            128
Length: 307, dtype: int64

In [37]:
print(combined_df.columns[combined_df.isnull().sum() > 1])

Index(['상장폐지일', '종업원1인당 부가가치증가율(IFRS)', '종업원수증가율(IFRS)', '종업원1인당 매출액증가율(IFRS)',
       '종업원1인당 인건비증가율(IFRS)', '사내유보율(IFRS)', '사내유보 대 자기자본비율(IFRS)',
       '평균배당률(IFRS)', '자기자본배당률(IFRS)', '배당성향(IFRS)', '1주당순자산(IFRS)(원)',
       '종업원1인당 부가가치(IFRS)(백만원)', '종업원1인당 매출액(IFRS)(백만원)',
       '종업원1인당 정상영업이익(IFRS)(백만원)', '종업원1인당 순이익(IFRS)(백만원)',
       '종업원1인당 인건비(IFRS)(백만원)', '노동장비율(IFRS)', '기계장비율(IFRS)', '자본집약도(IFRS)',
       '종업원1인당 부가가치(IFRS)(백만원).1', '종업원수(IFRS)'],
      dtype='object')


In [38]:
# 고쳐야 할 컬럼 리스트
columns_to_replace = ['종업원1인당 부가가치증가율(IFRS)', '종업원수증가율(IFRS)', '종업원1인당 매출액증가율(IFRS)',
                      '종업원1인당 인건비증가율(IFRS)', '사내유보율(IFRS)', '사내유보 대 자기자본비율(IFRS)',
                      '평균배당률(IFRS)', '자기자본배당률(IFRS)', '배당성향(IFRS)', '1주당순자산(IFRS)(원)',
                      '종업원1인당 부가가치(IFRS)(백만원)', '종업원1인당 매출액(IFRS)(백만원)',
                      '종업원1인당 정상영업이익(IFRS)(백만원)', '종업원1인당 순이익(IFRS)(백만원)',
                      '종업원1인당 인건비(IFRS)(백만원)', '노동장비율(IFRS)', '기계장비율(IFRS)', '자본집약도(IFRS)',
                      '종업원1인당 부가가치(IFRS)(백만원).1', '종업원수(IFRS)']

# 1. check2의 컬럼 이름에서 '_y' 제거
check2.columns = check2.columns.str.replace('_y', '')

for column in columns_to_replace:
    combined_df[column] = combined_df[column].mask(
        combined_df[column].isna(), check2[column]
    )

In [39]:
check2['종업원수(IFRS)']

0       150.0
1       134.0
2       526.0
3      5192.0
4      6382.0
        ...  
384     311.0
385     363.0
386     393.0
387     519.0
388     608.0
Name: 종업원수(IFRS), Length: 389, dtype: float64

In [40]:
print(combined_df.columns[combined_df.isnull().sum() > 1])

Index(['상장폐지일'], dtype='object')


전체 데이터 389중 컬럼별 0 개수 몇개인지 확인

In [41]:
pd.set_option('display.max_rows', None)

zero_count = (kospi_ratio==0).sum()
zero_count

회사명                              0
거래소코드                            0
회계년도                             0
소속코드                             0
통계청 한국표준산업분류 코드 10차(대분류)         0
산업코드                             0
산업명                              0
상장일                              0
상장폐지일                            0
총자본증가율(IFRS)                    28
유형자산증가율(IFRS)                   28
비유동생물자산증가율(IFRS)               300
투자부동산증가율(IFRS)                 198
비유동자산증가율(IFRS)                  28
유동자산증가율(IFRS)                   28
재고자산증가율(IFRS)                   48
자기자본증가율(IFRS)                   29
매출액증가율(IFRS)                    28
정상영업이익증가율(IFRS)                149
순이익증가율(IFRS)                   182
총포괄이익증가율(IFRS)                 159
종업원1인당 부가가치증가율(IFRS)             0
종업원수증가율(IFRS)                    0
종업원1인당 매출액증가율(IFRS)              0
종업원1인당 인건비증가율(IFRS)              0
매출액총이익률(IFRS)                    6
매출액정상영업이익률(IFRS)                 2
매출액순이익률(IFRS)                    0
총자본사업이익률(IFRS)      

In [42]:
filtered_zero_count = zero_count[zero_count < 150]
filtered_zero_count

회사명                              0
거래소코드                            0
회계년도                             0
소속코드                             0
통계청 한국표준산업분류 코드 10차(대분류)         0
산업코드                             0
산업명                              0
상장일                              0
상장폐지일                            0
총자본증가율(IFRS)                    28
유형자산증가율(IFRS)                   28
비유동자산증가율(IFRS)                  28
유동자산증가율(IFRS)                   28
재고자산증가율(IFRS)                   48
자기자본증가율(IFRS)                   29
매출액증가율(IFRS)                    28
정상영업이익증가율(IFRS)                149
종업원1인당 부가가치증가율(IFRS)             0
종업원수증가율(IFRS)                    0
종업원1인당 매출액증가율(IFRS)              0
종업원1인당 인건비증가율(IFRS)              0
매출액총이익률(IFRS)                    6
매출액정상영업이익률(IFRS)                 2
매출액순이익률(IFRS)                    0
총자본사업이익률(IFRS)                   0
총자본정상영업이익률(IFRS)                 0
총자본순이익률(IFRS)                    0
자기자본정상영업이익률(IFRS)                1
자기자본순이익률(IFRS)      

---
#### 중복 컬럼 제거

In [47]:
len(combined_df.columns)

157

In [46]:
columns_to_drop = [col for col in combined_df.columns if '.1' in col]
columns_to_drop

['부가가치(IFRS)(백만원).1',
 '종업원1인당 부가가치(IFRS)(백만원).1',
 '총자본투자효율(IFRS).1',
 '기계투자효율(IFRS).1',
 '부가가치율(IFRS).1']

In [48]:
combined_df.drop(columns=columns_to_drop, inplace = True)
len(combined_df.columns)

152

In [49]:
# combined_df.to_csv('../data/processed_data/재무비율_처리완.csv', index=False, encoding='UTF-8-sig')